# Introduction to flauers

Flauers (IPA: ˈflaʊərs', same pronunciation as "flowers") is a tool that allows the simulation of matrix-multiplication systolic arrays with seamless integration in pytorch for neural network evaluation. 

This tutorial will give you the basics of simulated fault injection using flauers.

First of all, we want to import the packages!

In [ ]:
import flauers

Uh oh! It seems like we cannot import the package! Something is missing... We forgot to install it!
Well, as of now, flauers does not provide any pip or conda package ready to be installed. The easyest thing is to add the path to your current python envirnoment. So, let's do it!

In [ ]:
import sys
sys.path.append("src/") # you always want to `append` the src path this repo
import flauers
# we print it just to verify it is working correctly.
print(flauers)

Great! Seems like everything went smoothly. 
We can try some matrix multiplication now

### Matrix Multiplication

In [ ]:
import numpy as np # We need it to intantiate the matrices 🤭

A = np.array([[1,2,3], [4,5,6], [7,8,9], [10, 11, 12]])
B = np.array([[1,2,3,4], [5,6,7,8], [9,10,11,12]])
print("A is\n",A, end="\n\n")
print("B is\n",B)

Ok, now that we have our matrices, let's try multipliyng them with embedded python BLAS routines:


In [ ]:
C = np.matmul(A, B) # equivalent to A@B
print(C)

Ok, let's try multiplying it with flauers now:

In [ ]:
flauers.matmul(A, B)

Pretty cool, right?!

So, now that we know how to do matrix multiplication let's go for some details.

## Data Types

The goal of flauers is to simulate some hardware at logic level.
Real hardware has buses and operators of fixed size. flauers models this aspect of the hardware through datatypes (also called dtypes).

As of now, only a subset of numpy dtypes are supported: int8, int16, int32, int64, float32, float64.
Nevertheless they should be enough for neural networks

When generating a `SystolicArray` object (more about that later) implicitly it is generated with int8 buses for the inputs.
Also take into account the fact that there are two main dtype: input dtypes and accumulation dtypes.

Anyways, let's try some floating points (more common in NNs)

In [ ]:
A_float = A.astype(np.float32)
B_float = B.astype(np.float32)
A_float # just to display it in jupyter

In [ ]:
B_float

As you can see, the matrices are now float32. 

Let's try as we did again:

In [ ]:
np.matmul(A_float, B_float)

In [ ]:
flauers.matmul(A_float, B_float)

Seems to work! How comes?

The answer is simple: even if A_float and B_float have a different dtype, it was possible to cast all the values to int8, and this conversion was performed implicitly. 

Let's try multiplying the matrices for 0.5 and do the multiplication again

In [ ]:
A_float = A * 0.5
A_float

In [ ]:
B_float = B*0.5
B_float

In [ ]:
A_float @ B_float

In [ ]:
flauers.matmul(A_float, B_float)

As you can see, an error was raised. A CastingError error in facts! Pretty self explanatory!
On top of that, additional informations are displayed, but we don't care about those.

We immediately see the problem: it says that it "couldnt convert A from float64 to int8". Perfect! 

So, what do we do? We can explicitly create a `SystolicArray` object with appropriate classes.

In [ ]:
hw = flauers.SystolicArray(10, 10, 10, flauers.projection_matrices.output_stationary, in_dtype=np.float32)
hw.matmul(A_float, B_float)

And this worked like a charm! 😎

There are some parameters that are "weird": what are all of those 10 and what is a projection matrix? 
There's quite a lot happening behind the scenes, so let's not care too much about everything. Refer to the documentation for details about that.

Nevertheless, here a quick explaination:

- The first three parameters (namely $N1, N2, N3$) define the size (in space and time) of the systolic array. Specifically, if we have matrices A (of dimensions $X, Y$) and B (of dimension $Y, Z$), we need to ensure that $N1 >= X$, $N3 >= Z$ and $N2 >= Y$. The reason behind it is due to the algorithm used to simulate the `SystolicArray`
- The projection matrice determines the shape of the systolic array. Specifically implying which registers are reused and how. 

The following figure shows the difference among three different systolic array. Even though they have the same $N1, N2, N3$ parameters, the projection matrix changes the shape and the way the data is forwarded.

In any case, you can find more details in the documentation and in [this paper](http://compalg.inf.elte.hu/~tony/Informatikai-Konyvtar/03-Algorithms%20of%20Informatics%201,%202,%203/Systolic30May.pdf)

> Please note that we always perform the computation $R = A \times B + C$ using $A$ as the activations, $B$ as the weights and $C$ as the _accumulation_ (which is both the added term $C$ and the partial sum of the matrix multiplication).
> Later on you will see that each processing element has three registers a, b and c which directly correspond to the values of matrices $A, B, C$.

> In the figures you can also see how a fault in a register affects the output matrix, depending on the injected register and the projection matrix

### No local reuse:
![nlr](misc/NoLocalReuse.drawio.png)

### Output Stationary:
![os](misc/OutputStationary.drawio.png)

### Row Stationary:
![rs](misc/RowStationary.drawio.png)

## Fault Injection

The whole point of flauers is to simulate Systolic Arrays and being able to perform fault injection on it. 
So, how do we perform fault injection? We simply create a `Fault` and we add it to the hardware. 

But first, we have to point out that the _processing elements_ (PEs) of the systolic array have all the same structure, which is the one in the following picture.

![basic_processing_element](misc/basic_processing_element.drawio.png)

When we create a fault, we have to choose which register to inject among `a`, `b` and `c`.

In [ ]:
"""
We want a fault on the first element of the array (0, 0) on register 'c' on bit 5. The fault is a stuckat and its polarity is high (1)
"""
f = flauers.fault_models.StuckAt(x=1, y=1, line="c", bit=5, polarity=1)
hw.add_fault(f) # we add the fault 

At this point we can perform the matmul with the fault

In [ ]:
hw.matmul(A_float, B_float)

We already see that the first element [0, 0] is not correct. This is the effect of the fault.

We can double check whether the result is the same with the `==` operator

In [ ]:
C_golden = A_float@B_float
C_faulty = hw.matmul(A_float, B_float)
C_golden == C_faulty

As predicted, only the first element is different! Great!

Nevertheless, I know it's obvious, but _repetita iuvant_: be careful when comparing with `==` and floating point. Minimal differences affect the result and could result in different outcomes.

Prefere comparing the difference like `C_golden-C_faulty < atol` or the relative difference `abs(C_golden-C_faulty)/abs(C_golden) < rtol`

> Hint: you can also use np.allclose(C_gloden, C_faulty) to compare the whole tensor at once.

Finally, it is possible to clear all the faults and restore the original functionality of the systolic array.

In [ ]:
hw.clear_all_faults()
hw.matmul(A_float, B_float)

In [ ]:
C_faulty = hw.matmul(A_float, B_float)
np.allclose(C_faulty, C_golden) # As you can see these are the same

## Convolution

Another common operation (especially if we want to use it for NNs) is the convolution. 

flauers implements a method for convolution that implements the convolution as a combination of some spatial transformation of the input matrices followed by a matrix multiplication. 
flauers calls implements this strategy to a class called lowlif (which stands for LOwering/LIFting). More details about that are given in the paper [caffe con troll](https://arxiv.org/abs/1504.04343)

Nevertheless, for now only one strategy is implemented which has the name of Image to Columns (abbreviated as Im2Col). Let's leave the details aside 🤭

Such method is implemented as a method of the package. Please note that it only supports squared matrices.

In [ ]:
A = np.array([[1,2,3], [4,5,6], [7,8,9]])
K = np.array( [[1, 1], [1, 1]])
C = flauers.convolve(A, K)
C

It is also possible to specify an explicit array

In [ ]:
C = flauers.convolve_with_array(A, K, array=hw)
C

And finally, it is possible to implement the strategy manually using the classes in the package

In [ ]:
# We need the input shapes to initialize the im2col object
im2col = flauers.lowerings.S_Im2Col( A.shape, K.shape )
A_low = im2col.lower_activation( A ) # we can transform the "activation"
K_low = im2col.lower_kernel( K ) # we can transform the kernel
print("A is:\n", A)
print("Lowered A is:\n", A_low)

print("K is:\n", K)
print("Lowered K is:\n", K_low)

C_low = hw.matmul(A_low, K_low)
C = im2col.lift(C_low)

print("Lowered C is:\n", C_low)
print("C is:\n", C)


# PyTorch

Let's see how we can use flauers for PyTorch. First of all, let's load a simple PyTorch model.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

## Model Loading

In [ ]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 6, 5, padding="valid")
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5, padding="valid")
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# for now map_location="cpu" is mandatory since flauers is not compatible yet with GPU
model = torch.load("best_model", map_location="cpu")
model

## Dataset Loading

This model is trained with LeNet5, so let's load it as well

In [ ]:
import torchvision
from torchvision.transforms import v2

pil_to_tensor = v2.Compose([
    # stransforms.RandomResizedCrop(size
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
])

mnist_test = torchvision.datasets.MNIST(".", download=True, train=False, transform=pil_to_tensor)
dataloader = torch.utils.data.DataLoader(mnist_test, batch_size = 512, shuffle=False, num_workers=4)

Let's try visualizing some samples, as usual

In [ ]:
from matplotlib import pyplot as plt

for i in range(9):
    img, label = mnist_test[i]

    # sugrid visualization (3 by 3)
    plt.subplot(3, 3, i+1)

    # show the image and label
    plt.imshow(img[0], cmap="grey")
    plt.title(label)

    # Deactivate the tick labels
    plt.xticks([], [])
    plt.yticks([], [])

# enhance the layout
plt.tight_layout()
plt.show()


## Baseline

Let's try to run the inference and compute the baseline accuracy of the model

In [ ]:
tot = 0
correct = 0
device = "cuda" if torch.cuda.is_available() else "cpu"


with torch.no_grad():
    for batch in dataloader:
        imgs, labels = batch
        outputs = model(imgs)

        tot += len(labels)
        correct += (outputs.argmax(1) == labels).sum()

    acc = correct / tot

print(f"model accuracy is {acc.item()*100:.2f}%")

## Fault Injection

Cool! We could see that the model accuracy is 99.13% for our model! That's great! 

But how does it behave when there are faults in the systolic array? Let's try some injections!

The first thing we need to do is to tell pytorch to use our hardware when invoking the convolution operation. 
Luckily for you, flauers offers a ready-to-use wrapper for PyTorch plus some utility functions to allow a smooth transition into the fault injection process.

flauers wrappers the hardware in a layer class, called `SystolicConvolution` and available through the `flauers.torch` package and has the same syntax as torch.nn.Conv2d.

Nevertheless, we rarely instantiate the layer by ourselves, since usually we have pre-trained models.
This means that usually we just want to subsitute a convolutional layer of another model. For this reason there are two usuful functions:

- ``flauers.torch.compatible_layers``
- ``faluers.torch.replace_layers``

Let's see how the process works

In [ ]:
import flauers.torch

print(model)

compatible_list = flauers.torch.compatible_layers(model)
print(compatible_list)

As you can see, the function takes the model as an input and spits out a list of `named_submodules` (cfr. torch documentation) that are compatible with flauers.

We can then **subsitute** a layer using the second function.
Bare in mind, the layer is subsitited in place, so generally we want to instantiate a second copy of the model.

In [ ]:
flauers.torch.replace_layers(model, compatible_list, hardware=hw)
print(model)

There it is. Our model is ready for fault injection.

To insert the injection we just need to invoke the method `hw.add_fault`

# Fault Injection Campaign

Let's see how to setup a fault injectioncampaign from scratch.

First of all, we need two copies of the same model: one will be our golden model, the other will be the faulty model implementing the systolic array.

In [ ]:
# First we instantiate the golden model twice
model = torch.load("best_model", map_location="cpu")
systolic_model = torch.load("best_model", map_location="cpu")

# We instantiate the hardware
hw = flauers.SystolicArray(
    29, 29, 161, # These values are fitting for the later torch experiments
    flauers.projection_matrices.output_stationary,
    in_dtype = np.dtype(np.float32)
)

# We then subsitute the layers, to simulate the systolic array doing its thing
compatible_layers = flauers.torch.compatible_layers(model)
flauers.torch.replace_layers(systolic_model, compatible_layers, hardware=hw)
print(model)
print(systolic_model)

Let us try an inference on the systolic model

In [ ]:
with torch.no_grad():
    correct = 0
    imgs, labels = batch
    output = systolic_model(imgs)
    correct += (outputs.argmax(1) == labels).sum()
    print(f"Systolic batch accuracy is {correct / output.shape[0] * 100:.2f}%")

As you see, there is a progressbar (implemented with tqdm) that shows the progress of the processing per batch per systolic array.
This means that for our model the progress bar will show up twice (becuase there are two systolic convolutions).

> Note: It is not possible to use flauers with the gradient computation (since the backward pass is not implemented), so please don't forget the statement `with torch.no_grad()`

## Helper classes

So, in this example we implemented some classes for ease of use of the tool.

In general, this is not needed but they highly simplify the injection process to almost completely automating it.

### Fault Generator

FaultGenerator is a class that automatically generates the parameters of a fault, given the space parameters. Also, it implements hotloading from a state dictionary that is written on the disk invoking the method `save_state`.
The method `get_fault` automatically generates a StuckAt fault in a random PE.
Finally, the [Leveugle formula from Date 2009 on statistical fault injection](https://ieeexplore.ieee.org/document/5090716) is directly implemented inside the class.

In [ ]:
import random

class FaultGenerator:
    def __init__(self, hw: flauers.SystolicArray,
                 name: None|str = None, automatic_save: bool = False, save_path: str = './_fidata', 
                 channels: int = 6, bits: int = 32, try_continue: bool = False
                ):
        self.hw = hw
        self.next_seed = 0
        self.channels = channels
        self.bits = bits
        self.possible_elements = list( self.hw.physical_mapping.keys() )

        if name is not None:
            self.name = name
        if automatic_save and name is None:
                raise Exception("You must give it a name to save the file")
        self.save_path = save_path
        self.automatic_save = automatic_save

        self._next_minus_one = True
        
        if try_continue:
            if name is None:
                raise Exception("You must give it a name to hot-reload")
            fn = os.path.join( save_path, f"FaultGenerator_{name}_state.pkl" )
            if not os.path.exists(fn):
                print(f"No old state file {fn} found. Starting from zero.")
            else:
                # print(f"Reloading state from {fn}.")
                self.load_state()

    def current_seed(self):
        return self.next_seed - 1
    
    def starting_point(self):
        return self.next_seed
    
    def statistical_faults(self,
                       error_margin: float = 0.05, 
                       confidence_level: float = 1.96,
                       error_probability: float = 0.5):
        # Population parameters
        n_channels = self.channels
        n_PEs = len(self.possible_elements)
        bits = self.bits
        
        lines = 3 # a, b, c - always
        polarities = 2 # 0 or 1 - always (with stuck-ats at least)

        # Total population
        N = n_channels * n_PEs * lines * bits * polarities

        # Statistical parameters
        e = error_margin
        t = confidence_level
        p = error_probability

        # Date 2009 Formula
        n = N / ( 1 + e**2 *  (N-1)/(t**2 * p * (1-p) ) )

        print(f"N is {N}. We need to inject {n} faults", end= " ")
        n_faults = int(np.ceil(n))
        print(f"which actually are {n_faults}")

        return n_faults

    def get_fault(self):
        random.seed(self.next_seed)
        line = random.choice(["a", "b", "c"])
        channel = random.randrange(0, self.channels)
        x, y = random.choice( self.possible_elements )
        bit = random.randrange(0, self.bits)
        polarity = random.randrange(0, 2)
        
        f = flauers.fault_models.StuckAt(
            line, 
            x = x, y = y,
            bit = bit, polarity = polarity
        )

        self._next_minus_one = True
        
        self.next_seed += 1
        if self.automatic_save:
            self.save_state()
            
        return (f, channel)

    def iteration_done(self):
        self._next_minus_one = False
        if self.automatic_save:
            self.save_state()

    def load_state(self, file_name: None|str = None, file_path: None|str = None):
        """ Check save_state to understand the parameters """
        if file_name is not None and not os.path.exists(file_path):
            raise Exception(f"Directory {file_path} does not exist.")

        if file_path is not None: path = file_path
        else: path = self.save_path

        if file_name is not None: path = os.path.join(path, f"{file_name}")
        elif self.name is not None: path = os.path.join(path, f"FaultGenerator_{self.name}_state.pkl")
        else: raise Exception(f"A name must be given!.")

        print(f"Reloading FaultGenerator {self.name if self.name is not None else ''} from file {path} ...", end=" ")
            
        file = open(path, "rb")
        state_dict = pickle.load(file)

        if "_next_minus_one" in state_dict:
            self._next_minus_one = state_dict["_next_minus_one"]
        else:
            self._next_minus_one = False
        
        self.next_seed = state_dict["next_seed"]
        if self._next_minus_one:
         self.next_seed -= 1

        print(f"Next seed will be {self.next_seed}")

        file.close()

    def save_state(self, file_name:None|str = None, file_path:None|str = None):
        state_dict = {
            "next_seed": self.next_seed,
            "_next_minus_one": self._next_minus_one
        }
        
        # The path is either the given or the default
        path = self.save_path if file_path is None else file_path
        if not os.path.exists(path):
            os.makedirs(path)

        # The name is the given parameter
        if file_name is not None: 
            path = os.path.join(path, f"FaultGenerator_{file_name}_state.pkl")
        # or the name given in the initializer
        elif self.name is not None:
            path = os.path.join(self.save_path, f"FaultGenerator_{self.name}_state.pkl")
        # if no name is provided either ways, there's an error.
        else:
            raise Exception("A name is necessary!")
            
        file = open(path, "wb")
        pickle.dump(state_dict, file)
        file.flush()
        file.close()
        

### Batch Loader

BatchLoader is a class that implements the automatic shuffling of the dataset with a fixed size. This is needed simply for the fact that, due to simulation timings, it is almost impossible to run the systolic array on the whole dataset. So the strategy in this case is to load a fixed number (`batch_size` to be precise) of images in random order for every injected fault.
So each batch will correspond to a different injected fault.

Please note that also this class has the option for saving and loading the state in case the experiments are abruptly interrupted for some reason.

In [ ]:
class BatchLoader:
    def __init__(self, dataset, #: torchvision.datasets.VisionDataset,
                 name: None|str = None, automatic_save: bool = False, save_path: str = './_fidata',  # fault injection data
                 batch_size = 64, try_continue: bool = False
                ):
        
        self.dataset = dataset
        self.hw = hw
        self.len_dataset = len(dataset)
        
        if name is not None:
            self.name = name
            
        self.automatic_save = automatic_save
        if automatic_save and name is None:
                raise Exception("You must give it a name to save the file")
        self.save_path = save_path

        if self.len_dataset % batch_size  != 0:
            raise Exception(f"""Please, select a batch-size multiple of the length of the dataset. 
            You selected {batch_size} The dataset is composed by {self.len_dataset} elements. 
            {self.len_dataset} is not divisible by {batch_size}""")

        self.seed = 0
        self.next_index = 0

        self._batch_size = batch_size
        self._next_minus_one = True
        
        random.seed(self.seed)
        self._order = [i for i in range( self.len_dataset )]
        random.shuffle(self._order)

        if not os.path.exists(self.save_path):
            os.makedirs(self.save_path)

        if try_continue:
            if name is None:
                raise Exception("You must give it a name to hot-reload")
            fn = os.path.join( save_path, f"BatchLoader_{self.name}_state.pkl" )
            if not os.path.exists(fn):
                print(f"No old state file {fn} found. Starting from zero.")
            else:
                # print(f"Reloading state from {fn}.")
                self.load_state()

    def current_index(self):
        if self._next_minus_one:
            return self.next_index - 1
        return self.next_index
        
    def current_seed(self):
        if self.next_index == 0:
            return self.seed - 1
        return self.seed
    
    def set_seed(self, seed):
        self.seed = seed
        self._order = [i for i in range( self.len_dataset )]
        random.seed(self.seed)
        random.shuffle(self._order)
    
    def load_state(self, file_name: None|str = None, file_path: None|str = None):
        """ Check save_state to understand the parameters """
        if file_name is not None and not os.path.exists(file_path):
            raise Exception(f"Directory {file_path} does not exist.")

        if file_path is not None: path = file_path
        else: path = self.save_path

        if file_name is not None: path = os.path.join(path, f"{file_name}")
        elif self.name is not None: path = os.path.join(path, f"BatchLoader_{self.name}_state.pkl")
        else: raise Exception(f"A name must be given!.")

        print(f"Reloading BatchLoader {self.name if self.name is not None else ''} from file {path}...", end=" " )
        
        file = open(path, "rb")
        state_dict = pickle.load(file)
        self.seed = state_dict["seed"]
        if "_next_minus_one" in state_dict:
            self._next_minus_one = state_dict["_next_minus_one"]
        else:
            self._next_minus_one = False
        self.next_index = state_dict["next_index"]
        if self._next_minus_one: 
            self.next_index -= 1
        if self.next_index < 0:
            self.next_index = 0
            self.seed -= 1
        self.batch_size = state_dict["batch_size"]
        self.len_dataset = state_dict["len_dataset"]

        print(f"Next index will be {self.next_index} with seed {self.seed}")
        
        file.close()

    def save_state(self, file_name: None|str = None, file_path: None|str = None):
        """
            Save the state for hot-reloading.
            If the object is initialized with a name, that will be used to save the state.
            Otherwise, file_name is mandatory to save the file. file_path is optional. If not given,
            the default ./_fidata will be used.
            
            Parameters
            ---
            file_name: [Optional] the name given to the state_file. It is mandatory if the object is initialized without a name
            file_path: [Optional] the path in which to save the file. If not given the default ./_fidata is used
        """
        state_dict = {
            "seed": self.seed,
            "next_index": self.next_index,
            "batch_size": self._batch_size,
            "len_dataset": self.len_dataset,
            "_next_minus_one": self._next_minus_one,
        }

        # The path is either the given or the default
        path = self.save_path if file_path is None else file_path
        if not os.path.exists(path):
            os.makedirs(path)

        # The name is the given parameter
        if file_name is not None: 
            path = os.path.join(path, f"BatchLoader_{file_name}_state.pkl")
        # or the name given in the initializer
        elif self.name is not None:
            path = os.path.join(self.save_path, f"BatchLoader_{self.name}_state.pkl")
        # if no name is provided either ways, there's an error.
        else:
            raise Exception("A name is necessary!")
            
        file = open(path, "wb")
        pickle.dump(state_dict, file)
        file.flush()
        file.close()

    def get_batch(self):
        img_shape = self.dataset[0][0].shape
        tot_batches = self.len_dataset / self._batch_size
        if self.next_index >= tot_batches: # let's make a new order
            self.seed += 1
            random.seed(self.seed)
            self._order = [i for i in range(self.len_dataset)]
            random.shuffle(self._order)
            self.next_index = 0

        starting_point = self.next_index * self._batch_size 
        imgs = np.zeros( (self._batch_size, *img_shape), dtype=np.float32)
        labels = np.zeros( self._batch_size, dtype=np.float32)
        for i in range(self._batch_size):
            idx = self._order[ i + starting_point ]
            img, label = self.dataset[ idx ]
            imgs[i, :, :] = img
            labels[i] = label
        self._next_minus_one = True
        self.next_index += 1
        if self.automatic_save:
            self.save_state()
        return ( torch.from_numpy(imgs), torch.from_numpy(labels) )

    def iteration_done(self):
        self._next_minus_one = False
        if self.automatic_save:
            self.save_state()

### Result Writer

ResultWriter implements the dumping of the results on a csv file. Ultimately, this is the result of the fault injection campaign. It contains the data gathered through the fault injection process. Also in this case, there is the option of saving and loading the state, to recover the experiments in case of abrupt stop

In [ ]:
import os, csv, pickle

class ResultsWriter():
    def __init__(self, file_path, 
                 try_continue = True,
                 fields = [ "fault_seed", "dataset_seed", "dataset_index",
                          "line", "channel", "row", "col", "bit", "polarity", "ftype"],
                ):
        self.file_path = file_path
        self.fields = fields

        newFile = False # This is needed to write the header
        
        # Open the file
        # There are 4 cases:
        #     1. try_continue = False and file exists -> raise exception.
        #     2. try_continue = True and file exists -> then we read the existing file
        #     3. try_continue = False and file not exists -> we make a new file
        #     4. try_continue = True and file not exists -> we create the file
        if os.path.exists(file_path) and not try_continue: # case 1.
            raise Exception(f"File {file_path} exists, but hot-reloading is disabled.")
        if try_continue and os.path.exists(file_path): # case 2.
            if not self._validate_csv(file_path): raise Exception(f"Fields are different!")
            self.file = open(file_path, "a")
        else: # case 3. and 4.
            parent = os.path.dirname(file_path)
            if not os.path.exists(parent):
                os.makedirs(parent)
            self.file = open(file_path, "w")
            newFile = True
            
        self.writer = csv.DictWriter(self.file, fieldnames = fields)
        if newFile:
            print(f"Initializing file {file_path}...", end ="")
            self.writer.writeheader()
            self.file.flush()
            print(f" Header wrote.")

    def write_row(self, data: list|dict, multiple_rows: bool = False):
        if multiple_rows:
            self.writer.writerows(data)
        else:
            self.writer.writerow(data)
        self.file.flush()

    def close():
        self.file.close()
    
    def _validate_csv(self, file_path):
        file = open(file_path, "r")
        csv_file = csv.DictReader(file)
        try:
            row = next(iter(csv_file))
        except StopIteration: # The file exists but it's empty
            return True
            
        read_fields = row.keys()
        file.close()

        for field in self.fields:
            if field not in read_fields:
                print(f"field {field} does not exist in the file.")
                return False
        return True
        

### Fault Injection Campaign

Finally we reach the core of the fault injection. This class runs a fault injection campaign using the classes previously defined and computes the metrics. 
This class only implements the "fault_type" class but can be easily extended to compute other metrics like SDC1 or SDC5.

Nevertheless, in theory it is possible to retrieve the metrics by saving the complete output of the network.

In [ ]:
from tqdm.auto import trange

fault_types_names = ["masked", "non-critical", "sdc1"]

class FaultInjectionCampaign:
    def __init__(self, gmodel, fmodel, dataset, hw, channels, bits,
                model_name: str, dataset_name: str, hw_name: str,
                faulty_layers: list, bs_size = 100):
        """
        THE LAYER SUBSTITUTION SHOULD BE ALREADY DONE!
        """
        self.golden_model = gmodel
        self.faulty_model = fmodel
        self.dataset = dataset
        self.hw = hw

        self.faulty_layers = faulty_layers

        name = f"{model_name}.{dataset_name}.{hw_name}"
        self.name = name
        self.batch_loader = BatchLoader( dataset, name=name, batch_size = bs_size,
                                        automatic_save=True, try_continue=True )
        self.fault_generator = FaultGenerator(hw, channels = channels, bits = bits,
                                              name=name, automatic_save=True, try_continue=True )
        self.results_writer = ResultsWriter( f"./results/{name}.csv", try_continue=True )

        self.golden_model.eval()
        self.faulty_model.eval()

    def run_golden_inference(self):
        return self.golden_model(self.dataset)

    def compute_fault_type(self, golden, faulty: np.ndarray):
        gout = golden
        fout = faulty
        
        bs = gout.shape[0]
        gmax, glabel = torch.max(gout, 1)
        fmax, flabel = torch.max(fout, 1)
        logits = np.array( torch.allclose(gout, fout, rtol=0.01) )
    
        # default is sdc1
        fault_types = 2 * np.ones((bs))
        # if the labels are the same, the fault is non-critical    
        fault_types[ glabel == flabel ] = 1
        # if the logits are the same, masked
        fault_types[ np.all(logits, axis=tuple([i for i in range(1, len(logits.shape))]) ) ] = 0
    
        return fault_types

    def run(self):
        
        n_faults = self.fault_generator.statistical_faults()
        start = self.fault_generator.starting_point()
        
        for i in trange(start, n_faults, dynamic_ncols=True, leave=True):
            # batch and fault generation
            batch = self.batch_loader.get_batch()
            fault, channel = self.fault_generator.get_fault()

            # fault instantiation
            for layer in self.faulty_layers:
                layer.clear_faults()
                layer.add_fault(fault, channel)

            # compute the inferences
            img, labels = batch
            with torch.no_grad():
                golden_output = self.golden_model(img)
                faulty_output = self.faulty_model(img)
    
            ftype = self.compute_fault_type(golden_output, faulty_output)

            res = []
            for i in range( len(ftype) ):
                res.append( {
                    "fault_seed": self.fault_generator.current_seed(), 
                    "dataset_seed": self.batch_loader.current_seed(),
                    "dataset_index": self.batch_loader.current_index(),
                    "line": fault.line.name,
                    "channel": channel,
                    "row": fault.x, 
                    "col": fault.y, # respectively row and column
                    "bit": fault.bit,
                    "polarity": fault.polarity,
                    "ftype": int(ftype[i]),
                } )

            self.results_writer.write_row(res, multiple_rows=True)
            self.batch_loader.iteration_done()
            self.fault_generator.iteration_done()

    def done(self):
        f = open(f"tmp.done.{self.name}", "w")
        f.write("DONE!\n")
        f.write(f"{datetime.now()}")
        f.flush()
        f.close()
        print("DONE!")

## Actual Fault injection

Using the helper classes defined before, we can simply call the run method of the FaultInjectionCampaign to run the whole thing.
Because we implemented the thing using tqdm, we have an estimation of the needed time for a fault injection campaign. 

> *Bonus*: since we implemented a mechanism for saving the state and reloading the state, it is possible to interrupt the process anytime and will resume from the last fault recorded! That's great in case of a crash of the system. Also, it is possible to disable the _hot-reloading_ by passing `False` to the `try_continue` and `automatic_save` parameters of the various objects.

In [ ]:
faulty_layers = [
    systolic_model.get_submodule("conv1"),
    systolic_model.get_submodule("conv2"),
]
fault_injection_campaign = FaultInjectionCampaign(
    model, systolic_model, mnist_test, hw, 6, 32, 
                                "lenet", "mnist", "os", faulty_layers, bs_size = 100
)
fault_injection_campaign.run()